In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests

## Data Collection

### Files sources

#### CityMart

In [2]:
# read "CityMart_data.csv"
df_city_mart=pd.read_csv("CityMart_data.csv", sep=",",index_col='item_code')
df_city_mart.head()

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified
item_code,,,,,,,,,,
P0019,CityMart,2.81,26.83,38.75,24.89,323,25877.199625,253,202,2023-01-19
P0024,CityMart,3.30,59.23,34.99,21.78,321,45138.128706,21,225,2023-01-24
P0025,CityMart,2.34,22.60,16.90,60.12,291,22962.232800,316,278,2023-01-25
P0034,CityMart,6.54,18.59,68.72,21.99,126,28092.330552,612,233,2023-02-03
P0039,CityMart,9.94,57.89,88.33,35.45,312,181270.870165,968,203,2023-02-08


#### Greenfield_Grocers

In [3]:
# read "Greenfield_Grocers_data.csv"

df_grennfield_grocers=pd.read_csv("Greenfield_Grocers_data.csv", sep="|",header=3,  index_col="ITEM_CODE" )
df_grennfield_grocers.drop([ "1",'Unnamed: 12'], axis=1, inplace=True)
df_grennfield_grocers.columns = [col.lower() for col in df_grennfield_grocers.columns]
df_grennfield_grocers.rename_axis(df_grennfield_grocers.index.name.lower(), inplace=True)
df_grennfield_grocers.head()

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified
item_code,,,,,,,,,,
P0006,Greenfield_Grocers,5.02,86.68,71.64,16.42,66,101964.180384,450,130,2023-01-06
P0014,Greenfield_Grocers,9.91,21.67,54.91,84.80,99,100903.494560,827,118,2023-01-14
P0016,Greenfield_Grocers,1.13,60.03,97.39,90.00,348,526168.953000,981,136,2023-01-16
P0021,Greenfield_Grocers,0.95,40.36,67.91,99.76,301,273426.956576,289,155,2023-01-21
P0028,Greenfield_Grocers,5.24,22.37,61.78,68.67,15,94903.217262,423,177,2023-01-28


#### Outlet_data

In [4]:
# read "SuperSaver_Outlet_data.xlsx"

df_outlet_data=  pd.read_excel("SuperSaver_Outlet_data.xlsx", sheet_name=None)
df_outlet_data
#df_outlet_data.set_index('item_code', inplace=True)
#df_outlet_data.head()

{'Quantity':     item_code  quantity_sold
 0       P0003            198
 1       P0007            211
 2       P0008            200
 3       P0009            209
 4       P0012            186
 ..        ...            ...
 374     P1976            236
 375     P1980            192
 376     P1986            193
 377     P1993            185
 378     P2000            250
 
 [379 rows x 2 columns],
 'Info':     Unnamed: 0          item code  store name   mass  dimension length  \
 0        P0003  SuperSaver_Outlet        2.40  50.90              8.80   
 1        P0007  SuperSaver_Outlet        9.88  43.65             48.64   
 2        P0008  SuperSaver_Outlet        1.45  36.18             10.23   
 3        P0009  SuperSaver_Outlet        6.98  48.13             45.55   
 4        P0012  SuperSaver_Outlet        7.20  23.66             65.90   
 ..         ...                ...         ...    ...               ...   
 374      P1976  SuperSaver_Outlet        5.10  62.99             49

In [5]:
df_outlet_data['Info'].drop('stock age', axis=1, inplace=True)
df_outlet_data

{'Quantity':     item_code  quantity_sold
 0       P0003            198
 1       P0007            211
 2       P0008            200
 3       P0009            209
 4       P0012            186
 ..        ...            ...
 374     P1976            236
 375     P1980            192
 376     P1986            193
 377     P1993            185
 378     P2000            250
 
 [379 rows x 2 columns],
 'Info':     Unnamed: 0          item code  store name   mass  dimension length  \
 0        P0003  SuperSaver_Outlet        2.40  50.90              8.80   
 1        P0007  SuperSaver_Outlet        9.88  43.65             48.64   
 2        P0008  SuperSaver_Outlet        1.45  36.18             10.23   
 3        P0009  SuperSaver_Outlet        6.98  48.13             45.55   
 4        P0012  SuperSaver_Outlet        7.20  23.66             65.90   
 ..         ...                ...         ...    ...               ...   
 374      P1976  SuperSaver_Outlet        5.10  62.99             49

In [6]:
df_outlet_data['Info'].columns=["item_code",
    "store_name",
    "mass",
    "dimension_length",
    "dimension_width",
    "dimension_height",
    "days_since_last_purchase",
    "package_volume",
    "stock_age"
]

df_outlet_data=pd.merge(df_outlet_data['Quantity'], df_outlet_data['Info'], left_on = 'item_code' , right_on = 'item_code')

df_outlet_data=df_outlet_data[['item_code',
    "store_name",
    "quantity_sold",
    "mass",
    "dimension_length",
    "dimension_width",
    "dimension_height",
    "days_since_last_purchase",
    "package_volume",
    "stock_age"]]

df_outlet_data.set_index('item_code', inplace=True)
df_outlet_data.head()

,store_name,quantity_sold,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age
item_code,,,,,,,,,
P0003,SuperSaver_Outlet,198,2.40,50.90,8.80,68.82,337,30825.854400,277
P0007,SuperSaver_Outlet,211,9.88,43.65,48.64,55.78,16,118428.526080,291
P0008,SuperSaver_Outlet,200,1.45,36.18,10.23,69.14,315,25590.193596,169
P0009,SuperSaver_Outlet,209,6.98,48.13,45.55,44.66,295,97909.078190,443
P0012,SuperSaver_Outlet,186,7.20,23.66,65.90,12.05,40,18788.287700,45


#### HighStreet_Bazaar

In [7]:
# read 'HighStreet_Bazaar_data.json'
df_highstreet_bazar=pd.read_json('HighStreet_Bazaar_data.json', orient='records')
df_highstreet_bazar.set_index('item_code', inplace=True)
df_highstreet_bazar
df_highstreet_bazar.head()

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified
item_code,,,,,,,,,,
P0001,HighStreet_Bazaar,6.11,75.46,91.62,92.08,78.0,636608.450016,237,346,1672531200000
P0011,HighStreet_Bazaar,4.34,16.97,93.21,54.58,344.0,86333.208546,184,218,1673395200000
P0015,HighStreet_Bazaar,1.37,58.93,11.28,73.87,344.0,49103.634648,946,315,1673740800000
P0017,HighStreet_Bazaar,7.27,51.51,39.21,16.17,138.0,32658.663807,268,228,1673913600000
P0020,HighStreet_Bazaar,0.89,57.50,69.84,6.12,333.0,24576.696000,396,228,1674172800000


In [8]:
data = pd.concat([df_city_mart, df_grennfield_grocers, df_outlet_data,df_highstreet_bazar], axis=0)
data

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified
item_code,,,,,,,,,,
P0019,CityMart,2.81,26.83,38.75,24.89,323.0,25877.199625,253,202,2023-01-19
P0024,CityMart,3.30,59.23,34.99,21.78,321.0,45138.128706,21,225,2023-01-24
P0025,CityMart,2.34,22.60,16.90,60.12,291.0,22962.232800,316,278,2023-01-25
P0034,CityMart,6.54,18.59,68.72,21.99,126.0,28092.330552,612,233,2023-02-03
P0039,CityMart,9.94,57.89,88.33,35.45,312.0,181270.870165,968,203,2023-02-08
...,...,...,...,...,...,...,...,...,...,...
P1978,HighStreet_Bazaar,6.28,72.48,47.71,83.54,322.0,288883.057632,637,235,1843344000000
P1981,HighStreet_Bazaar,0.91,31.13,55.22,71.22,342.0,122427.080292,60,241,1843603200000
P1982,HighStreet_Bazaar,4.65,65.72,82.01,60.75,192.0,327424.104900,925,237,1843689600000


#### Aggregate

#### Simple baseline

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def get_simple_baseline(data, fillna_value=-1, drop_cols=None, k_fold=5, scaler='standard', model='linear', metric='mae', target_col=None, X_data_test=None):
    
    data = data.copy()
    # Handle missing values
    data.fillna(fillna_value, inplace=True)
    if X_data_test is not None:
        X_data_test = X_data_test.copy()
        X_data_test.fillna(fillna_value, inplace=True)
    
    # Drop unwanted columns
    if drop_cols:
        data.drop(drop_cols, axis=1, inplace=True)
        if X_data_test is not None:
            X_data_test.drop(drop_cols, axis=1, inplace=True)

    # Split data into features (X) and target (y)
    y = data[target_col]
    X = data.drop(target_col, axis=1)

    # Feature scaling
    if scaler == 'standard':
        scaler = StandardScaler()
    elif scaler == 'minmax':
        scaler = MinMaxScaler()
    else:
        scaler = None
    
    if scaler:
        X = scaler.fit_transform(X)
        if X_data_test is not None:
            X_data_test = scaler.transform(X_data_test)

    # Initialize the model
    if model == 'linear':
        model = LinearRegression()
    elif model == 'logistic':
        model = LogisticRegression()
    elif model == 'random_forest':
        model = RandomForestClassifier()
    else:
        raise ValueError("Unsupported model type")

    # Initialize cross-validation
    kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)
    scores = []

    # Train and evaluate using k-fold cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate using the specified metric
        if metric == 'mae':
            score = mean_absolute_error(y_test, y_pred)
        elif metric == 'accuracy':
            score = accuracy_score(y_test, np.round(y_pred))
        else:
            raise ValueError("Unsupported metric")

        scores.append(score)

    if X_data_test is not None:
        model.fit(X, y)
        return np.mean(scores), model.predict(X_data_test)
    
    # Return the average score
    return np.mean(scores)

In [10]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

np.float64(45.04056766700087)

### API sources

In [11]:
def get_api(endpoint_url):
    try:
        # Make the GET request to the mock API
        response = requests.get(endpoint_url)

        if response.status_code == 200:
            data = response.json()
            print(data["message"])
            return data['data']
        else:
            print(f"Failed to retrieve volume data. Status code: {response.status_code}")
    
    except Exception as e:
        print(f"An error occurred: {e}")
password = get_api("https://www.raphaelcousin.com/api/exercise/auth")["password"]
prices = get_api(f"https://www.raphaelcousin.com/api/exercise/{password}/prices")


Authentication successful
Volume data retrieved successfully


In [12]:
df_prices =pd.DataFrame.from_dict(prices, orient='index', columns=['prices'])
df_prices

,prices
P0001,22.14
P0002,26.91
P0003,16.90
P0004,7.04
P0005,20.84
...,...
P1996,25.80
P1997,26.05
P1998,17.41
P1999,7.57


#### Aggregate

In [13]:
data = pd.merge(data, df_prices, left_index=True, right_index=True, how='left')
data

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified,prices
item_code,,,,,,,,,,,
P0019,CityMart,2.81,26.83,38.75,24.89,323.0,25877.199625,253,202,2023-01-19,17.86
P0024,CityMart,3.30,59.23,34.99,21.78,321.0,45138.128706,21,225,2023-01-24,14.04
P0025,CityMart,2.34,22.60,16.90,60.12,291.0,22962.232800,316,278,2023-01-25,3.38
P0034,CityMart,6.54,18.59,68.72,21.99,126.0,28092.330552,612,233,2023-02-03,15.11
P0039,CityMart,9.94,57.89,88.33,35.45,312.0,181270.870165,968,203,2023-02-08,14.20
...,...,...,...,...,...,...,...,...,...,...,...
P1978,HighStreet_Bazaar,6.28,72.48,47.71,83.54,322.0,288883.057632,637,235,1843344000000,15.85
P1981,HighStreet_Bazaar,0.91,31.13,55.22,71.22,342.0,122427.080292,60,241,1843603200000,17.98
P1982,HighStreet_Bazaar,4.65,65.72,82.01,60.75,192.0,327424.104900,925,237,1843689600000,17.75


In [14]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

np.float64(44.10144429622423)

### Scrapping sources

!pip install selenium

In [15]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver (e.g., Chrome)
driver = webdriver.Chrome()  # Make sure ChromeDriver is installed
# driver = webdriver.Firefox()
# driver = webdriver.Edge()
# driver = webdriver.Safari()

# Open the URL
url = 'https://www.raphaelcousin.com/module4/scrapable-data'
driver.get(url)

# Wait for the page to fully load (increase time if needed)
time.sleep(5)

# Get the fully rendered page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Initialize lists to store scraped data
exercise_data = []

# Find both tables
tables = soup.find_all('table')

# Close the Selenium WebDriver
driver.quit()

# Scrape the second table (Exercise Data)
course_table = tables[1]
for row in course_table.find('tbody').find_all('tr'):
    cols = row.find_all('td')
    exercise_data.append({ 'item_code':cols[0].text ,
    'Customer Score': int(cols[1].text),
    'Total Reviews':int(cols[2].text)
     })
 




# Convert the lists to pandas DataFrames
df_exercise = pd.DataFrame(exercise_data)
df_exercise


,item_code,Customer Score,Total Reviews
0,P0001,2,972
1,P0002,3,260
2,P0003,2,285
3,P0004,5,512
4,P0005,3,85
...,...,...,...
1995,P1996,5,512
1996,P1997,2,989
1997,P1998,2,440
1998,P1999,4,37


In [16]:
df_exercise.set_index('item_code', inplace=True)
df_exercise.head()

,Customer Score,Total Reviews
item_code,,
P0001,2,972
P0002,3,260
P0003,2,285
P0004,5,512
P0005,3,85


#### Aggregate

In [17]:
data = pd.merge(data, df_exercise, left_index=True, right_index=True, how='left')
data.head()

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified,prices,Customer Score,Total Reviews
item_code,,,,,,,,,,,,,
P0019,CityMart,2.81,26.83,38.75,24.89,323.0,25877.199625,253,202,2023-01-19,17.86,1,948
P0024,CityMart,3.30,59.23,34.99,21.78,321.0,45138.128706,21,225,2023-01-24,14.04,3,436
P0025,CityMart,2.34,22.60,16.90,60.12,291.0,22962.232800,316,278,2023-01-25,3.38,5,703
P0034,CityMart,6.54,18.59,68.72,21.99,126.0,28092.330552,612,233,2023-02-03,15.11,2,935
P0039,CityMart,9.94,57.89,88.33,35.45,312.0,181270.870165,968,203,2023-02-08,14.20,2,188


In [18]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

np.float64(40.02769508950211)

### Generating Submission File

In [19]:
# X_test =  read  Neighborhood_Market_data

# read
df_StoreN =  pd.read_csv("Neighborhood_Market_data.csv", sep=",", index_col='item_code')
df_StoreN

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,last_modified
item_code,,,,,,,,,
P0002,Neighborhood_Market,5.51,51.79,46.72,72.02,344,174261.666176,287,2023-01-02
P0004,Neighborhood_Market,3.97,84.63,39.42,42.46,189,141651.425916,387,2023-01-04
P0005,Neighborhood_Market,5.99,39.33,83.51,5.12,183,16816.375296,382,2023-01-05
P0010,Neighborhood_Market,4.10,77.43,49.56,74.41,208,285543.225828,656,2023-01-10
P0013,Neighborhood_Market,6.96,95.39,34.61,23.24,114,76725.649196,755,2023-01-13
...,...,...,...,...,...,...,...,...,...
P1972,Neighborhood_Market,5.68,17.89,71.87,95.67,296,123008.113881,411,2028-05-25
P1977,Neighborhood_Market,1.53,84.32,64.91,66.30,171,362873.902560,702,2028-05-30
P1991,Neighborhood_Market,7.79,84.46,82.96,52.14,50,365334.635424,190,2028-06-13


In [20]:
df_StoreN = pd.merge(df_StoreN, df_prices, left_index=True, right_index=True, how='left')
df_StoreN = pd.merge(df_StoreN, df_exercise, left_index=True, right_index=True, how='left')
df_StoreN 

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,last_modified,prices,Customer Score,Total Reviews
item_code,,,,,,,,,,,,
P0002,Neighborhood_Market,5.51,51.79,46.72,72.02,344,174261.666176,287,2023-01-02,26.91,3,260
P0004,Neighborhood_Market,3.97,84.63,39.42,42.46,189,141651.425916,387,2023-01-04,7.04,5,512
P0005,Neighborhood_Market,5.99,39.33,83.51,5.12,183,16816.375296,382,2023-01-05,20.84,3,85
P0010,Neighborhood_Market,4.10,77.43,49.56,74.41,208,285543.225828,656,2023-01-10,10.00,2,709
P0013,Neighborhood_Market,6.96,95.39,34.61,23.24,114,76725.649196,755,2023-01-13,12.98,3,984
...,...,...,...,...,...,...,...,...,...,...,...,...
P1972,Neighborhood_Market,5.68,17.89,71.87,95.67,296,123008.113881,411,2028-05-25,15.88,2,854
P1977,Neighborhood_Market,1.53,84.32,64.91,66.30,171,362873.902560,702,2028-05-30,16.21,1,991
P1991,Neighborhood_Market,7.79,84.46,82.96,52.14,50,365334.635424,190,2028-06-13,15.32,1,572


In [21]:
 _, x_pred = get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold', X_data_test = df_StoreN)

In [22]:
_,x_pred

(np.float64(40.02769508950211),
 array([160.69791741, 244.75235981, 159.97446231, 227.49906711,
        241.32803691, 199.15468505, 181.89867296, 184.82447983,
        196.01119833, 195.93514238, 225.56770118, 200.57373072,
        227.85006606, 216.89160291, 231.56978156, 205.72713312,
        205.99021879, 177.17330025, 230.95072376, 205.02198978,
        216.86309161, 164.67185805, 245.94539069, 186.53312434,
        210.04947981, 160.33533258, 294.69489181, 210.76937999,
        163.48276227, 175.50869395, 213.75270967, 244.03043876,
        180.94061676, 193.70159209, 238.26273334, 216.67167061,
        204.64310589, 184.69598662, 253.48650797, 191.95866868,
        197.47338964, 170.5081523 , 179.43702396, 154.23158107,
        181.13125676, 200.6007    , 275.42957861, 197.69668842,
        182.85776304, 191.05223086, 232.40245984, 204.15891691,
        216.85044109, 173.05951501, 215.37876993, 212.63204961,
        167.4375182 , 219.82974481, 210.16810078, 154.4190366 ,
        

In [23]:
submission = pd.DataFrame({
    'item_code': df_StoreN.index,
    'quantity_sold': x_pred # your_prediction
})

submission.to_csv('submission.csv', index=False, sep=',')
submission.head()

,item_code,quantity_sold
0,P0002,160.697917
1,P0004,244.752360
2,P0005,159.974462
3,P0010,227.499067
4,P0013,241.328037
